# The validation of the generalizability of adversarial samples

We have defined three network models, deep maxout network model, shallow softmax network model, and shallow RBF network model. We use the maxout network model to generate adversarial examples on the MNIST dataset, and then use the shallow softmax network model and shallow RBF network model to detect the generated adversarial examples.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

class MaxoutLayer(layers.Layer):
    """
    Customizing the Maxout Layer
    """
    def __init__(self, units, num_pieces, **kwargs):
        super(MaxoutLayer, self).__init__(**kwargs)
        self.units = units
        self.num_pieces = num_pieces

    def call(self, inputs):
        # Reshape the input to (batch_size, units, num_pieces)
        inputs = tf.reshape(inputs, (-1, self.units, self.num_pieces))
        # Get the maximum value in each group
        return tf.reduce_max(inputs, axis=-1)

def create_maxout_model(input_shape, num_classes, units=256, num_pieces=2):
    """
    Create a Maxout network model, using a custom Maxout
    :param input_shape: shape of the input data
    :param num_classes: Number of output categories
    :param units: Number of neuron units
    :param num_pieces: Maxout Number of slices per cell
    :return: Maxout network model
    """
    model = models.Sequential()
    model.add(layers.Dense(units * num_pieces, input_shape=input_shape))
    model.add(MaxoutLayer(units, num_pieces))
    model.add(layers.Flatten())
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# load MNIST data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Pre-processed data
x_train = x_train.reshape((x_train.shape[0], -1)).astype('float32') / 255
x_test = x_test.reshape((x_test.shape[0], -1)).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Creating a Maxout model
maxout_model = create_maxout_model(input_shape=(784,), num_classes=10)

# compilation model
maxout_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training Models
maxout_model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))


11490434/11490434 [==============================] - 0s 0us/step
Epoch 1/5
469/469 [==============================] - 8s 4ms/step - loss: 0.2495 - accuracy: 0.9292 - val_loss: 0.1221 - val_accuracy: 0.9639
Epoch 2/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0993 - accuracy: 0.9711 - val_loss: 0.0920 - val_accuracy: 0.9730
Epoch 3/5
469/469 [==============================] - 2s 5ms/step - loss: 0.0668 - accuracy: 0.9798 - val_loss: 0.0826 - val_accuracy: 0.9740
Epoch 4/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0491 - accuracy: 0.9854 - val_loss: 0.0768 - val_accuracy: 0.9760
Epoch 5/5
469/469 [==============================] - 2s 4ms/step - loss: 0.0373 - accuracy: 0.9887 - val_loss: 0.0765 - val_accuracy: 0.9762


In [2]:
def create_softmax_model(input_shape, num_classes):
    """
    Creating a shallow Softmax network model
    :param input_shape: Shape of input data
    :param num_classes: Number of output categories
    :return: Softmax network model
    """
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Creating a shallow Softmax model
softmax_model = create_softmax_model(input_shape=(784,), num_classes=10)

# compilation model
softmax_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training Models
softmax_model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))


Epoch 1/5
469/469 [==============================] - 3s 5ms/step - loss: 0.6691 - accuracy: 0.8360 - val_loss: 0.3824 - val_accuracy: 0.9009
Epoch 2/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3620 - accuracy: 0.9021 - val_loss: 0.3184 - val_accuracy: 0.9129
Epoch 3/5
469/469 [==============================] - 2s 3ms/step - loss: 0.3196 - accuracy: 0.9115 - val_loss: 0.2995 - val_accuracy: 0.9162
Epoch 4/5
469/469 [==============================] - 2s 3ms/step - loss: 0.2996 - accuracy: 0.9165 - val_loss: 0.2850 - val_accuracy: 0.9209
Epoch 5/5
469/469 [==============================] - 2s 3ms/step - loss: 0.2878 - accuracy: 0.9205 - val_loss: 0.2781 - val_accuracy: 0.9228


In [3]:
class RBFLayer(layers.Layer):
    def __init__(self, units, gamma, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma

    def build(self, input_shape):
        self.centers = self.add_weight(name='centers',
                                       shape=(self.units, input_shape[1]),
                                       initializer='uniform',
                                       trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, inputs):
        # Expanding the dimensions of inputs and centers for easier computation
        expanded_inputs = tf.expand_dims(inputs, 1)  # Shape: (batch_size, 1, num_features)
        expanded_centers = tf.expand_dims(self.centers, 0)  # Shape: (1, units, num_features)
        # Calculate the Euclidean distance
        distances = tf.reduce_sum(tf.square(expanded_inputs - expanded_centers), axis=-1)
        # Applying RBF Functions
        return tf.exp(-self.gamma * distances)

def create_rbf_model(input_shape, num_classes, units=128, gamma=1.0):
    model = models.Sequential()
    model.add(RBFLayer(units, gamma, input_shape=input_shape))
    model.add(layers.Dense(num_classes, activation='softmax'))
    return model

# Creating shallow RBF models
rbf_model = create_rbf_model(input_shape=(784,), num_classes=10, units=512, gamma=0.05)

# compilation model
rbf_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training Models
rbf_model.fit(x_train, y_train, epochs=5, batch_size=128, validation_data=(x_test, y_test))




Epoch 1/5
469/469 [==============================] - 9s 18ms/step - loss: 1.7971 - accuracy: 0.4989 - val_loss: 1.2182 - val_accuracy: 0.7462
Epoch 2/5
469/469 [==============================] - 7s 16ms/step - loss: 0.8295 - accuracy: 0.8284 - val_loss: 0.5953 - val_accuracy: 0.8690
Epoch 3/5
469/469 [==============================] - 7s 16ms/step - loss: 0.5256 - accuracy: 0.8786 - val_loss: 0.4465 - val_accuracy: 0.8970
Epoch 4/5
469/469 [==============================] - 8s 18ms/step - loss: 0.4260 - accuracy: 0.8985 - val_loss: 0.3762 - val_accuracy: 0.9114
Epoch 5/5
469/469 [==============================] - 7s 16ms/step - loss: 0.3700 - accuracy: 0.9094 - val_loss: 0.3340 - val_accuracy: 0.9198


In [4]:
# Functions for creating adversarial samples
def create_adversarial_pattern(input_image, input_label, model, epsilon=0.1):
    input_image = tf.convert_to_tensor(input_image, dtype=tf.float32)
    input_label = tf.convert_to_tensor(input_label, dtype=tf.float32)
    loss_object = tf.keras.losses.CategoricalCrossentropy()
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = loss_object(input_label, prediction)
    gradient = tape.gradient(loss, input_image)
    signed_grad = tf.sign(gradient)
    perturbations = epsilon * signed_grad
    return perturbations

# Generating Adversarial Samples
epsilon = 0.25  # The size of the perturbation
perturbations = create_adversarial_pattern(x_train, y_train, maxout_model, epsilon=epsilon)
adv_x_train = x_train + perturbations.numpy()

# Evaluating model performance on adversarial samples
adv_loss, adv_accuracy = maxout_model.evaluate(adv_x_train, y_train)
print(f'Adversarial Loss: {adv_loss}, Adversarial Accuracy: {adv_accuracy}')


1875/1875 [==============================] - 5s 3ms/step - loss: 53.7344 - accuracy: 0.0019
Adversarial Loss: 53.73444747924805, Adversarial Accuracy: 0.0019166666315868497


In [5]:
# Evaluating the Performance of Softmax Models on Adversarial Samples
adv_loss_softmax, adv_accuracy_softmax = softmax_model.evaluate(adv_x_train, y_train)
print(f'Softmax Model - Adversarial Loss: {adv_loss_softmax}, Adversarial Accuracy: {adv_accuracy_softmax}')

# Evaluating the performance of RBF models on adversarial samples
adv_loss_rbf, adv_accuracy_rbf = rbf_model.evaluate(adv_x_train, y_train)
print(f'RBF Model - Adversarial Loss: {adv_loss_rbf}, Adversarial Accuracy: {adv_accuracy_rbf}')


1875/1875 [==============================] - 5s 3ms/step - loss: 13.5632 - accuracy: 0.0196
Softmax Model - Adversarial Loss: 13.56315803527832, Adversarial Accuracy: 0.019600000232458115
1875/1875 [==============================] - 7s 3ms/step - loss: 2.1784 - accuracy: 0.1821
RBF Model - Adversarial Loss: 2.178393840789795, Adversarial Accuracy: 0.1820833384990692
